In [32]:
import dill as pickle
import pandas as pd
import subprocess
import os
import sys
sys.path.append("../fun")
from fun import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
infile = open("../model/all_setting_results.pkl",'rb')
all_setting_results = pickle.load(infile)
infile.close()

In [3]:
infile = open("../features/all_datasets_for_modeling.pkl",'rb')
all_datasets_for_modeling = pickle.load(infile)
infile.close()

In [4]:
infile = open("../features/build_train_test_for_model_parameters.pkl",'rb')
build_train_test_for_model_parameters = pickle.load(infile)
infile.close()

In [51]:
list(all_setting_results["201908230535"].keys())

['count_global_1.0',
 'count_global_2.0',
 'count_global_2.1',
 'registered_global_1.0',
 'registered_global_2.0',
 'registered_global_2.1',
 'casual_global_1.0',
 'casual_global_2.0',
 'casual_global_2.1',
 'count_workingday.0_1.0',
 'count_workingday.0_2.0',
 'count_workingday.0_2.1',
 'count_workingday.1_1.0',
 'count_workingday.1_2.0',
 'count_workingday.1_2.1',
 'registered_workingday.0_1.0',
 'registered_workingday.0_2.0',
 'registered_workingday.0_2.1',
 'registered_workingday.1_1.0',
 'registered_workingday.1_2.0',
 'registered_workingday.1_2.1',
 'casual_workingday.0_1.0',
 'casual_workingday.0_2.0',
 'casual_workingday.0_2.1',
 'casual_workingday.1_1.0',
 'casual_workingday.1_2.0',
 'casual_workingday.1_2.1']

In [46]:
g = correct_dic_results("201908230535",build_train_test_for_model_parameters["target"]["subs"])

In [48]:
for k,v in g.items():
    for j,y in v.items():
        for m in y:
            flatlist = flatten(m)
            print(k,j,flatlist)

1.0 global ['registered', 'casual']
1.0 global ['count']
1.0 workingday [('registered', '0'), ('registered', '1'), ('casual', '0'), ('casual', '1')]
1.0 workingday [('count', '0'), ('count', '1')]
2.0 global ['registered', 'casual']
2.0 global ['count']
2.0 workingday [('registered', '0'), ('registered', '1'), ('casual', '0'), ('casual', '1')]
2.0 workingday [('count', '0'), ('count', '1')]
2.1 global ['registered', 'casual']
2.1 global ['count']
2.1 workingday [('registered', '0'), ('registered', '1'), ('casual', '0'), ('casual', '1')]
2.1 workingday [('count', '0'), ('count', '1')]


In [40]:
def flatten(nested_lst):
    """ Return a list after transforming the inner lists
        so that it's a 1-D list.

    >>> flatten([[[],["a"],"a"],[["ab"],[],"abc"]])
    ['a', 'a', 'ab', 'abc']
    """
    if not isinstance(nested_lst, list):
        return(nested_lst)

    res = []
    for l in nested_lst:
        if not isinstance(l, list):
            res += [l]
        else:
            res += flatten(l)


    return(res)

In [34]:
def arrange_list(maliste,subs):
    
    if type(maliste[0]) is tuple:
        istuple = True
    else:
        istuple = False
        
    
    mybool = []
    if istuple:
        uniquewords = list(set([item[0] for item in maliste]))
        for u in uniquewords:
            idx = [idu for idu,item in enumerate(maliste) if item[0]==u]
            mybool.append(idx)
        L = []
        for indices in mybool:
            L.append([item for idi,item in enumerate(maliste) if idi in indices])

        newmaliste = [item[0][0] for item in L]

        newboolsubs = [idx for idx,item in enumerate(newmaliste) if item in subs]
        newbooltarget = [idx for idx,item in enumerate(newmaliste) if item not in subs]
        bigbool = [newboolsubs,newbooltarget]
        final = []
        for indices in bigbool:
            final.append([item for idx,item in enumerate(L) if idx in indices])

    else:
        uniquewords = list(set(maliste))
        for u in uniquewords:
            idx = [idu for idu,item in enumerate(maliste) if item==u]
            mybool.append(idx)
        L = []
        for indices in mybool:
            L = L + [item for idi,item in enumerate(maliste) if idi in indices]

        newmaliste = L

        newboolsubs = [idx for idx,item in enumerate(newmaliste) if item in subs]
        newbooltarget = [idx for idx,item in enumerate(newmaliste) if item not in subs]
        bigbool = [newboolsubs,newbooltarget]
        final = []
        for indices in bigbool:
            final.append([item for idx,item in enumerate(L) if idx in indices])

    return final

In [45]:
def correct_dic_results(runid,subs): 
    
    d = {}
    for k,v in all_setting_results[runid].items():
        strate,value,idmodel = k.split("_")

        if idmodel in d:
            d[idmodel].append((strate,value))
        else:
            d[idmodel] = [(strate,value)]

    x = {}
    for k,v in d.items():
        z = {}
        for t in v:

            split = t[1].split(".")
            if split[0] in z:
                if len(split)==1:
                    z[split[0]].append(t[0])
                else:
                    z[split[0]].append((t[0],split[1]))
            else:
                if len(split)==1:
                    z[split[0]] = [t[0]]
                else:
                    z[split[0]] = [(t[0],split[1])]

        x[k] = z


    for idrun,v in x.items():
        for strate in v:
            corrected = arrange_list(v[strate],subs)
            v[strate] = corrected

    return x